In [1]:
import equinox as eqx
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt
import numpy as np
import optax
from datasets import load_dataset
import jax.random as random
from jax.random import PRNGKey, split

from aevb.core import AEVB


# Data Processing Functions ----------------------------------
def one_hot_encode(x, k):
    "Create a one-hot encoding of x of size k."
    return jnp.array(x[:, None] == jnp.arange(k), dtype=jnp.float32)


@jax.jit
def prepare_data(X):
    num_examples = X.shape[0]
    num_pixels = 28 * 28
    X = X.reshape(num_examples, num_pixels)
    X = X / 255.0

    return X, num_examples


def data_stream(seed, data, batch_size, data_size):
    """Return an iterator over batches of data."""
    rng = np.random.RandomState(seed)
    num_batches = int(jnp.ceil(data_size / batch_size))
    while True:
        perm = rng.permutation(data_size)
        for i in range(num_batches):
            batch_idx = perm[i * batch_size : (i + 1) * batch_size]
            yield data[batch_idx]


# Generative Model and Recognition Feature Extractor --------------------
# class GenModel(eq.Module):
#     @nn.compact
#     def __call__(self, x, train: bool = False):
#         x = nn.Dense(features=128)(x)
#         x = nn.BatchNorm(use_running_average=not train)(x)
#         x = nn.relu(x)
#         x = nn.Dense(features=256)(x)
#         x = nn.relu(x)
#         x = nn.Dense(784)(x)
#         return x


# class RecModel(nn.Module):
#     latent_dim: int

#     @nn.compact
#     def __call__(self, x, train: bool = False):
#         x = nn.Dense(features=512)(x)
#         x = nn.BatchNorm(use_running_average=not train)(x)
#         x = nn.relu(x)
#         x = nn.Dense(features=256)(x)
#         x = nn.relu(x)
#         x = nn.Dense(features=128)(x)
#         x = nn.relu(x)
#         x = nn.Dense(features=64)(x)
#         z_mu = nn.Dense(features=self.latent_dim)(x)
#         z_logvar = nn.Dense(features=self.latent_dim)(x)
#         z_sigma = jnp.exp(z_logvar * 0.5)
#         return z_mu, z_sigma



/home/pscemama/aevb/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
class MlpEncoder(eqx.Module):
    #spectral_linear: eqx.nn.SpectralNorm[eqx.nn.Linear]
    linear1: eqx.nn.Linear
    linear2: eqx.nn.Linear

    def __init__(self, key):
        key1, key2, key3, key4 = random.split(key, 4)
        #self.norm1 = eqx.nn.BatchNorm(input_size=3, axis_name="batch")
        # self.spectral_linear = eqx.nn.SpectralNorm(
        #     layer=eqx.nn.Linear(in_features=3, out_features=32, key=key1),
        #     weight_name="weight",
        #     key=key2,
        # )
        #self.norm2 = eqx.nn.BatchNorm(input_size=32, axis_name="batch")
        self.linear1 = eqx.nn.Linear(in_features=10, out_features=32, key=key3)
        self.linear2 = eqx.nn.Linear(in_features=32, out_features=3, key=key4)

    def __call__(self, x, state):
        #x, state = self.norm1(x, state)
        #x, state = self.spectral_linear(x, state)
        x = jax.nn.relu(x)
        #x, state = self.norm2(x, state)
        x = self.linear1(x)
        x = jax.nn.relu(x)
        x = self.linear2(x)
        return x, state


class MlpDecoder(eqx.Module):
    #spectral_linear: eqx.nn.SpectralNorm[eqx.nn.Linear]
    linear1: eqx.nn.Linear
    linear2: eqx.nn.Linear

    def __init__(self, key):
        key1, key2, key3, key4 = random.split(key, 4)
        #self.norm1 = eqx.nn.BatchNorm(input_size=3, axis_name="batch")
        # self.spectral_linear = eqx.nn.SpectralNorm(
        #     layer=eqx.nn.Linear(in_features=3, out_features=32, key=key1),
        #     weight_name="weight",
        #     key=key2,

        #self.norm2 = eqx.nn.BatchNorm(input_size=32, axis_name="batch")
        self.linear1 = eqx.nn.Linear(in_features=3, out_features=8, key=key3)
        self.linear2 = eqx.nn.Linear(in_features=8, out_features=10, key=key4)

    def __call__(self, x, state):
        #x, state = self.norm1(x, state)
        #x, state = self.spectral_linear(x, state)
        x = jax.nn.relu(x)
        #x, state = self.norm2(x, state)
        x = self.linear1(x)
        x = jax.nn.relu(x)
        x = self.linear2(x)
        return x, state

from aevb.core import wrap_eqx_model


In [5]:
# Main Function --------------------------------
def main(save_samples_pth: str):

    # Prepare Data
    mnist_data = load_dataset("mnist")
    data_train = mnist_data["train"]

    X_train = np.stack([np.array(example["image"]) for example in data_train])
    X_train, N_train = prepare_data(X_train)

    seed = 1
    n = N_train.item()
    batch_size = 100
    batches = data_stream(seed, X_train, batch_size, n)

    # Create AEVB inference engine
    latent_dim = 3
    optimizer = optax.adam(1e-3)

    init, step, sample_data = AEVB(
        latent_dim=latent_dim,
        generative_model=(MlpEncoder, {}),
        recognition_model=(MlpDecoder, {}),
        optimizer=optimizer,
        n_samples=15,
        nn_lib="equinox",
    )

    # Run AEVB
    key = PRNGKey(1242)
    num_steps = 10000
    eval_every = 100

    key, init_key = split(key)
    state = init(init_key, X_train.shape[-1])

    key, *training_keys = split(key, num_steps + 1)
    for i, rng_key in enumerate(training_keys):
        batch = next(batches)
        state, info = step(rng_key, state, batch)
        if i % eval_every == 0:
            print(f"Step {i} | loss: {info.loss} | nll: {info.nll} | kl: {info.kl}")

    # Random Data Samples of Learned Generative Model
    key, data_samples_key = split(key)
    samples, _ = sample_data(data_samples_key, state.gen_params, state.gen_state, 5)
    fig, axs = plt.subplots(5, 1)
    for i, s in enumerate(samples):
        axs[i].imshow(s.reshape(28, 28))
    plt.savefig(save_samples_pth, format="png")


if __name__ == "__main__":
    from time import localtime, strftime

    now = strftime("%Y-%m-%d %H:%M:%S", localtime())
    main(f"./samples-{now}.png")


2024-03-24 19:13:30.405629: W external/xla/xla/service/gpu/nvptx_compiler.cc:742] The NVIDIA driver's CUDA version is 12.0 which is older than the ptxas CUDA version (12.4.99). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


TypeError: 'tuple' object is not callable

In [14]:
def foo(x, **kwargs):
    bar(x, **kwargs)

def bar(x, **kwargs):
    print(x)
    print(kwargs)

_in = (1, {"2": 2})
foo(_in)

(1, {'2': 2})
{}


SyntaxError: can't use starred expression here (2566220079.py, line 1)